In [3]:
import numpy as np

class ConvolutionalLayer:
    def __init__(self, n_input_channels, m_output_channels, kernel_size, stride, padding):
        self.n_input_channels = n_input_channels
        self.m_output_channels = m_output_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.weights = np.random.randn(m_output_channels, n_input_channels, kernel_size, kernel_size)
        self.biases = np.zeros((m_output_channels, 1))
        self.output = None
        self.input = None
        self.d_weights = np.zeros(self.weights.shape)
        self.d_biases = np.zeros(self.biases.shape)
        
    def conv_forward(self, input_data):
        n, c, h, w = input_data.shape
        out_h = (h - self.kernel_size + 2 * self.padding) // self.stride + 1
        out_w = (w - self.kernel_size + 2 * self.padding) // self.stride + 1
        padded_input = np.pad(input_data, ((0,0), (0,0), (self.padding, self.padding), (self.padding, self.padding)), 'constant')
        self.output = np.zeros((n, self.m_output_channels, out_h, out_w))
        self.input = input_data
        for i in range(n):
            for j in range(self.m_output_channels):
                for k in range(out_h):
                    for l in range(out_w):
                        self.output[i, j, k, l] = np.sum(padded_input[i, :, k * self.stride:k * self.stride + self.kernel_size, l * self.stride:l * self.stride + self.kernel_size] * self.weights[j, :, :, :]) + self.biases[j]
        return self.output
    
    def conv_backward(self, d_output):
        n, c, h, w = self.input.shape
        d_input = np.zeros(self.input.shape)
        padded_d_input = np.pad(d_input, ((0,0), (0,0), (self.padding, self.padding), (self.padding, self.padding)), 'constant')
        for i in range(n):
            for j in range(self.m_output_channels):
                for k in range(h):
                    for l in range(w):
                        padded_d_input[i, :, k:k + self.kernel_size, l:l + self.kernel_size] += self.weights[j, :, :, :] * d_output[i, j, k // self.stride,l // self.stride]
                        
        self.d_weights = np.zeros(self.weights.shape)
        self.d_biases = np.zeros(self.biases.shape)
        for i in range(n):
            for j in range(self.m_output_channels):
                for k in range(self.kernel_size):
                    for l in range(self.kernel_size):
                        self.d_weights[j, :, k, l] = np.sum(self.input[i, :, k:k + h, l:l + w] * d_output[i, j, :, :], axis=(0, 1))
        self.d_biases = np.sum(d_output, axis=(0, 2, 3))
        padded_d_input = padded_d_input[:, :, self.padding:-self.padding, self.padding:-self.padding]
        return d_input
    
    
    def conv_flatten(self, input):
        n, c, h, w = input.shape
        output = input.reshape(n, c * h * w)
        return output



In [4]:
# Defining the input image shape

# n = number of images in the tensor.

# c = number of channels in each image ( grayscale image has 1 channel, RGB image has 3 channels (red, green, and blue))

# h = height of each image.

# w = width of each image.

n, c, h, w = 1, 1, 28, 28

# Defining the convolutional layer with kernel size 3, stride 1 and padding 1
conv_layer = ConvolutionalLayer(n_input_channels=1, m_output_channels=10, kernel_size=3, stride=1, padding=1)

# Defining the input image
input_image = np.random.randn(n, c, h, w)

# Printing the shape of the input
print("Input Shape:", input_image.shape)

# Passing the input image through the convolutional layer
output = conv_layer.conv_forward(input_image)

# Printing the shape of the output
print("Output Shape:", output.shape)

# Flattening the convolved image
flattened_output = conv_layer.conv_flatten(output)
print("Flattened Shape: ", flattened_output.shape)

Input Shape: (1, 1, 28, 28)
Output Shape: (1, 10, 28, 28)
Flattened Shape:  (1, 7840)
